# Array Multicore

Instead of trivially parallel independent tasks here we want to use multiple threads to process simultaneously different parts of the same array. `dask` automatically provides this feature by replacing the `numpy` function with `dask` functions. The key concept is a chunk, each chunk of data is executed separately by different threads. For example for a matrix we define a 2D block size and each of those blocks can be executed independently and then the results accumulated to get to the final answer. See <http://dask.pydata.org/>

In [1]:
import numpy as np
import dask.array as da

In [15]:
A = np.random.rand(20000,20000)

In [16]:
A.size / 1e6

400.0

In [33]:
from distributed import Client
client = Client('127.0.0.1:8786')

In [34]:
%time B = A**2 + np.sin(A) * A * np.log(A)

CPU times: user 20.4 s, sys: 1.73 s, total: 22.1 s
Wall time: 22.2 s


In [35]:
A_dask = da.from_array(A, chunks=(2000, 2000))

In [36]:
A_dask.numblocks

(10, 10)

In [37]:
%time B_dask = (A_dask**2 + da.sin(A_dask) * A_dask * da.log(A_dask)).compute()

CPU times: user 3.92 s, sys: 4.41 s, total: 8.32 s
Wall time: 38.8 s


In [25]:
assert np.allclose(B, B_dask)

In [32]:
(A_dask**2 + da.sin(A_dask) * A_dask * da.log(A_dask)).visualize()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
